# Demo de "First Order Motion Model for Image Animation"  para animar Caras (se muestran todas las de "sample") con el mismo video
fuente: https://github.com/AliaksandrSiarohin/first-order-model


1) Preparar el entorno:

In [1]:
#@title Clonar paquete first-order-model
!git clone https://github.com/AliaksandrSiarohin/first-order-model
print("\nFirst Order Motion Model  instalado:")
%cd first-order-model
!ls .

Cloning into 'first-order-model'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 302 (delta 0), reused 0 (delta 0), pack-reused 299
Receiving objects: 100% (302/302), 72.15 MiB | 17.56 MiB/s, done.
Resolving deltas: 100% (155/155), done.

First Order Motion Model  instalado:
/content/first-order-model
animate.py	 demo.ipynb	    logger.py	       requirements.txt
augmentation.py  demo.py	    modules	       run.py
config		 Dockerfile	    old_demo.ipynb     sup-mat
crop-video.py	 frames_dataset.py  README.md	       sync_batchnorm
data		 LICENSE.md	    reconstruction.py  train.py


In [18]:
#@title Instalar paquete 'imageio-ffmpeg'
!pip install imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.2 MB/s 


In [2]:
#@title Cargar Librerías
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
from demo import make_animation
from skimage import img_as_ubyte
warnings.filterwarnings("ignore")

print("Librerías cargadas.")

Librerías cargadas.


In [11]:
#@title Montar el drive
from google.colab import drive
drive.mount('/content/gdrive')

dir = '/content/gdrive/MyDrive/IA/pruebasDeepFake'
subdir_ImagenesyVideos = 'demoCaras' #@param {type:"string"}
subdir_Modelos = 'checkpoints-models' #@param {type:"string"}
dirDatos = dir + '/' + subdir_ImagenesyVideos + '/'
dirModelos = dir + '/' + subdir_Modelos + '/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
#@title Cargar checkpoint del modelo 'first-order-mode' ya entrenado

from demo import make_animation

# redefine porque sino falla por no usar "yaml.safe_load"
#from demo import load_checkpoints as fom_load_checkpoints
import yaml
from modules.generator import OcclusionAwareGenerator
from modules.keypoint_detector import KPDetector
from animate import normalize_kp
from scipy.spatial import ConvexHull
from sync_batchnorm import DataParallelWithCallback
def nfom_load_checkpoints(config_path, checkpoint_path, cpu=False):

    with open(config_path) as f:
        config = yaml.safe_load(f) # cambio

    generator = OcclusionAwareGenerator(**config['model_params']['generator_params'],
                                        **config['model_params']['common_params'])
    if not cpu:
        generator.cuda()

    kp_detector = KPDetector(**config['model_params']['kp_detector_params'],
                             **config['model_params']['common_params'])
    if not cpu:
        kp_detector.cuda()
    
    if cpu:
        checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    else:
        checkpoint = torch.load(checkpoint_path)
 
    generator.load_state_dict(checkpoint['generator'])
    kp_detector.load_state_dict(checkpoint['kp_detector'])
    
    if not cpu:
        generator = DataParallelWithCallback(generator)
        kp_detector = DataParallelWithCallback(kp_detector)

    generator.eval()
    kp_detector.eval()
    
    return generator, kp_detector


print("Librerías cargadas de 'first-order-mode'.")


generator, kp_detector = nfom_load_checkpoints(config_path = '/content/first-order-model/config/vox-256.yaml', 
                            checkpoint_path = dirModelos + 'vox-cpk.pth.tar')

print("Modelo 'first-order-mode' cargado.")

Librerías cargadas de 'first-order-mode'.
Modelo 'first-order-mode' cargado.


In [15]:
#@title Definir funciones auxiliares
# función para mostrar imagen con video
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

def procesa(imagenFuente, videoFuente):
    source_image = imageio.imread(imagenFuente)
    driving_video = imageio.mimread(videoFuente, memtest=False)

    #Resize image and video to 256x256
    source_image = resize(source_image, (256, 256))[..., :3]
    driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

    predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                                adapt_movement_scale=True)

    return HTML(display(source_image, driving_video, predictions).to_html5_video())

print("Funciones auxiliares definidas.")    

Funciones auxiliares definidas.


2) Procesar las imágenes:


In [16]:
import os

video_seleccionado =  "sample2-dicaprio.mp4" #@param [ 'sample1-obama.mp4', 'sample2-dicaprio.mp4', 'jackNicholson.mp4' ] {allow-input: true}
videoFuente = dirDatos + video_seleccionado

arDir = os.listdir(dirDatos)
imagenes = []
for ar in arDir:
  if ar.find('.png')>=0 or ar.find('.jpg')>=0 or ar.find('.jpeg')>=0:
    if ar.find('sample')>=0:
      imagenes.append( ar )

print(videoFuente)
print(imagenes)

/content/gdrive/MyDrive/IA/pruebasDeepFake/demoCaras/sample2-dicaprio.mp4
['sample1-shrek.png', 'sample2-chucky.png', 'sample3-it.png', 'sample4-socrates.png', 'sample5-putin.png', 'sample6-downeyjr.png', 'sample9.png', 'sample7-wryder.png', 'sample8.png']


In [19]:
procesa(dirDatos + imagenes[0], videoFuente)

100%|██████████| 444/444 [00:18<00:00, 24.24it/s]


In [20]:
procesa(dirDatos + imagenes[1], videoFuente)

100%|██████████| 444/444 [00:18<00:00, 23.47it/s]


In [21]:
procesa(dirDatos + imagenes[2], videoFuente)

100%|██████████| 444/444 [00:16<00:00, 27.02it/s]


In [22]:
procesa(dirDatos + imagenes[3], videoFuente)

100%|██████████| 444/444 [00:16<00:00, 26.78it/s]


In [23]:
procesa(dirDatos + imagenes[4], videoFuente)

100%|██████████| 444/444 [00:16<00:00, 26.79it/s]
